# Prefix Tuning
이번 실습시간에는 Generative Model을 통한 분류 문제 해결을 위하여 Prefix tuning을 사용해보겠습니다.

P* tuning은 Prompt tuning, P-tuning, Prefix tuning을 아우르는 용어로, 이 학습법들을 적용하기 위해선 HuggingFace의 [PEFT]("https://github.com/huggingface/peft")(Parameter-Efficient Fine-tuning)라이브러리가 필요합니다.

## 필요한 요소 준비 및 불러오기

### 라이브러리 불러오기

In [1]:
from transformers import AutoModelForCausalLM
from peft import get_peft_model, PrefixTuningConfig, TaskType
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset

/home/elicer/Day2/day2-1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 모델 불러오기

이번 시간에 활용할 모델은 Bigscience의 BLOOMZ입니다. BLOOM을 zero-shot에 적응하도록 Fine-tuning 한 모델이며 다개국어에 능하다는 장점이 있습니다.

In [67]:
model_id = "bigscience/bloomz-560m"

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Prefix Tuning
Prefix tuning은 기존의 사전 훈련된 언어 모델의 파라미터를 동결하고, 소수의 새로운 파라미터만을 학습하여 특정 태스크에 적합하게 모델을 최적화하는 방법입니다. 연속적인 태스크-특화 벡터(Prefix, 접두사)를 학습하고, 이 벡터를 입력 시퀀스의 시작 부분에 추가합니다. 이 벡터는 가상의 토큰처럼 작용하여, 모델의 다음 토큰들이 이 프리픽스에 주의(attention)를 기울이게 합니다. 이러한 방식으로, Prefix는 기존 모델에 새로운 '문맥' 또는 '지시자'를 제공하여, 모델이 주어진 태스크에 더 적합한 출력을 생성할 수 있도록 돕습니다.`PrefixTuningConfig`를 통해 설정할 수 있습니다. 

In [68]:
peft_config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM,
                                 num_virtual_tokens=30)

`num_virtual_tokens`: 사용할 Soft token의 수로, 학습 대상이기에 수가 늘어날 수록 모델의 학습 리소스와 속도에 영향을 미칠 수 있습니다.

### 데이터셋
이번 실습에 사용할 데이터셋은 트위터에서 수집된 불만 사항들을 포함하고 있으며, 텍스트와 이에 대한 레이블로 구성되어있습니다. 일반적인 분류 태스크는 Classifier가 출력부에 있는 모델(BERT)을 사용하지만, 여기에선 일반적인 LLM을 통하여 분류 작업을 수행하는 것을 목표로 합니다. 데이터셋은 소수의 학습데이터와 다수의 테스트 데이터로 구성되기에, 적은 수의 데이터로 모델 학습을 극대화 하는 것이 제한 조건입니다.

In [3]:
dataset = load_dataset("ought/raft", "twitter_complaints")
dataset

DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 3399
    })
})

### 텍스트 데이터셋 전처리 하이퍼 파라미터

데이터셋을 전처리하기 전 사용할 파라미터를 정의합니다.

In [70]:
text_column = "Tweet text"
label_column = "text_label"
max_length = 64
lr = 3e-2
num_epochs = 40
batch_size = 4

분류 작업을 위해 데이터셋에 텍스트로 된 레이블을 추가합니다. 텍스트 생성을 통한 분류에서는 숫자 카테고리를 출력하기 보다 텍스트 상으로 카테고리를 말하게 만드는 것이 목표입니다. 레이블은 불만/불만 없음/미분류 세 가지로 나뉩니다.

In [71]:
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
print(classes)

['Unlabeled', 'complaint', 'no complaint']


In [72]:
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=4,
)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 3399
    })
})


분리된 데이터셋을 아래와 같이 확인해보겠습니다.

In [73]:
dataset["train"][1]

{'Tweet text': '@KristaMariePark Thank you for your interest! If you decide to cancel, you can call Customer Care at 1-800-NYTIMES.',
 'ID': 1,
 'Label': 2,
 'text_label': 'no complaint'}

### 토큰화

데이터 전처리 및 모델링을 위해 토크나이저를 불러오고 토큰화를 수행합니다. 시퀀스의 길이를 일정하게 맞추기 위해 패딩을 적용하며, 종료토큰 또한 첨부하여 문장의 끝을 알립니다. 

In [74]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token_id is None:  # 패딩 적용
    tokenizer.pad_token_id = tokenizer.eos_token_id
    
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
print(target_max_length)    # 각 클래스 레이블을 토크나이저로 변환한 후, 생성된 input_ids의 길이를 계산하여 그 중 최대값을 찾습니다. 

3


### 데이터 전처리 함수
데이터셋의 각 예제를 모델이 처리할 수 있는 형식으로 변환합니다. 이 과정에는 토큰화, 입력값 조정, 패딩, 텐서 변환 등의 단계가 포함됩니다.

In [75]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]   # 입력 데이터 설정
    targets = [str(x) for x in examples[label_column]]  # 레이블 문자열 변환
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets, add_special_tokens=False)
    
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i] # 토큰화 적용
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]

        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
        
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * ( # 패딩 적용
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[  # Attention 연산을 위한 마스크 적용
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])  # 텐서 변환
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 전처리 일괄 적용
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset: 100%|██████████| 3399/3399 [00:00<00:00, 9360.77 examples/s]


### 데이터셋 구성

In [76]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["train"]

### 데이터 로더 구성
Torch tensor 형태의 데이터를 학습시키기 위해 데이터로더를 구성합니다. 

In [77]:
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

- `shuffle=True`: 학습 데이터로더에서는 각 에폭마다 데이터를 무작위로 섞어서 모델이 데이터의 순서에 의존하지 않게 합니다. 이는 모델의 일반화 능력을 향상시키는 데 도움이 됩니다.
- `collate_fn=default_data_collator`: 배치 데이터를 어떻게 처리할지 정의하는 함수입니다. `default_data_collator`는 딕셔너리 형태의 데이터 포인트들을 자동으로 적절히 묶어 텐서로 변환합니다.
- `pin_memory=True`: 데이터를 GPU에 더 빠르게 전송할 수 있게 해주는 테크닉입니다. 특히 CUDA와 함께 사용할 때, 호스트 메모리에서 핀된(고정된) 메모리를 통해 더 빠른 데이터 전송이 가능해집니다.

테스트 데이터에도 동일한 전처리를 수행합니다.

In [78]:
def test_preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    model_inputs = tokenizer(inputs)
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
    return model_inputs


test_dataset = dataset["test"].map(
    test_preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset: 100%|██████████| 3399/3399 [00:00<00:00, 13544.18 examples/s]


변환된 데이터를 살펴보겠습니다.

In [79]:
test_dataloader = DataLoader(test_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
next(iter(test_dataloader))

{'input_ids': tensor([[     3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
          227985,   5484,    915,   2566,  74757,  64626,  12384,  44639,    613,
           52282,   2670,  79920,   3344,   1002,    368,  17646,  14472,   8348,
             664,    718,      4,  19036,     17,  31849,     17,   6312,     76,
              44,  62470,     56,     91,     50,  14839,     21,  77658,    915,
             210],
         [     3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3, 227985,   5484,    915,    405, 187059,
            2256,    664,   2550,  18833,  18607, 162467,      4, 

- `input_ids`: 토큰화된 입력 데이터의 ID
- `attention_mask`: 모델이 실제 데이터와 패딩을 구분할 수 있게 해주는 배열. 1은 self-attention 연산의 대상이고, 0은 패딩된 부분이므로 연산 대상에서 제외

In [80]:
len(train_dataloader), len(test_dataloader)

(13, 850)

### 모델 불러오기
지정된 모델에 Prefix tuning을 적용하여 불러오고, 파라미터의 수를 조회합니다.

In [81]:
# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(model_id)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 1,474,560 || all params: 560,689,152 || trainable%: 0.26299064191632515


모델에 Prefix tuning이 제대로 적용되었는지 확인합니다.

In [82]:
model.peft_config

{'default': PrefixTuningConfig(peft_type=<PeftType.PREFIX_TUNING: 'PREFIX_TUNING'>, auto_mapping=None, base_model_name_or_path='bigscience/bloomz-560m', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=30, token_dim=1024, num_transformer_submodules=1, num_attention_heads=16, num_layers=24, encoder_hidden_size=1024, prefix_projection=False)}

### 옵티마이저
모델을 학습시키기 위한 옵티마이저 알고리즘과 학습률 스케줄러를 정의합니다.

- `AdamW`: Adam 알고리즘의 변형으로, 가중치 감쇠(weight decay)를 다루는 방식이 특징입니다. 기존의 L2 정규화 대신에 별도의 가중치 감소 단계를 사용하여, 학습 과정에서 과적합을 방지하고 일반화 성능을 향상시키는 데 도움을 줍니다.
- `get_linear_schedule_with_warmup`: 학습 초기에 학습률을 점진적으로 증가시킨 후, 일정 기간이 지나면 선형적으로 감소시키는 방식을 취합니다.

In [83]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

## 학습
모델 학습과 검증을 진행합니다.

In [84]:
model = model.to(device)
# 학습
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
# 검증
    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 13/13 [00:00<00:00, 36.43it/s]


epoch=0: train_ppl=tensor(9.6118, device='cuda:0') train_epoch_loss=tensor(2.2630, device='cuda:0') eval_ppl=tensor(1.3034, device='cuda:0') eval_epoch_loss=tensor(0.2650, device='cuda:0')


100%|██████████| 13/13 [00:00<00:00, 36.69it/s]


epoch=1: train_ppl=tensor(1.3974, device='cuda:0') train_epoch_loss=tensor(0.3346, device='cuda:0') eval_ppl=tensor(1.2812, device='cuda:0') eval_epoch_loss=tensor(0.2478, device='cuda:0')


100%|██████████| 13/13 [00:00<00:00, 36.70it/s]


...

epoch=37: train_ppl=tensor(1.0000, device='cuda:0') train_epoch_loss=tensor(2.7632e-06, device='cuda:0') eval_ppl=tensor(1.0000, device='cuda:0') eval_epoch_loss=tensor(2.9023e-06, device='cuda:0')


100%|██████████| 13/13 [00:00<00:00, 36.28it/s]


epoch=38: train_ppl=tensor(1.0000, device='cuda:0') train_epoch_loss=tensor(2.6868e-06, device='cuda:0') eval_ppl=tensor(1.0000, device='cuda:0') eval_epoch_loss=tensor(2.8824e-06, device='cuda:0')


100%|██████████| 13/13 [00:00<00:00, 35.54it/s]

epoch=39: train_ppl=tensor(1.0000, device='cuda:0') train_epoch_loss=tensor(2.6723e-06, device='cuda:0') eval_ppl=tensor(1.0000, device='cuda:0') eval_epoch_loss=tensor(2.8740e-06, device='cuda:0')


### 평가
모델의 테스트 데이터셋 중 일부를 바탕으로 성능을 시각화해보겠습니다. 모델 출력부에 `"Label: "`로 시작하는 부분은 생성된 텍스트이지만, 마치 분류 모델의 Classification 결과처럼 작동하고 있습니다. 이 부분만 파싱하여 분류 작업을 prefix를 통해 수행할 수 있습니다. 아주 적은 수의 파라미터를 동원하여 모델이 생성 모델에서 분류를 수행할 수 있는 생성 모델로 전환이 된 셈입니다.

In [85]:
model.eval()
i = 16  # 16개 데이터에 대해서만 평가 수행

inputs = tokenizer(f'{text_column} : {dataset["test"][i]["Tweet text"]} Label : ', return_tensors="pt")
print(dataset["test"][i]["Tweet text"])
print(inputs)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10, eos_token_id=3
    )
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

Hey @nytimes your link to cancel my subscription isn't working and nobody is answering the chat. Please don't play that kind of stupid game.
{'input_ids': tensor([[227985,   5484,    915,  54078,   2566,   7782,  24502,   2632,   8989,
            427,  36992,   2670, 140711,  21994,  10789,    530,  88399,    632,
         183542,    368,  44799,     17,  29901,   5926,   7229,    861,  11596,
            461,  78851,  14775,     17,  77658,    915,    210]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[227985,   5484,    915,  54078,   2566,   7782,  24502,   2632,   8989,
            427,  36992,   2670, 140711,  21994,  10789,    530,  88399,    632,
         183542,    368,  44799,     17,  29901,   5926,   7229,    861,  11596,
            461,  78851,  14775,     17,  77658,    915,    210,   1936, 106863,
              2,   1936, 106863,      2,  16449,   5952,      2,  164